# Goal

We want to see how many ORFs in hg38 when lifted over to panTro3
remain a potential ORF. The strategy is simple, lift hg38
ribotricer index (after converting it to bed) to panTro3.
This lifted over bed file after accounting for half interval
based coordinates in the bed can be matched up to the ribotricer panTro3
index

In [1]:
import pandas as pd
import os
import re

In [2]:
hg38_ribotricer_index = '/home/cmb-06/as/skchoudh/genomes/hg38/ribotricer_v96_annotation_longest_candidate_orfs.tsv'
pantro3_ribotricer_index = '/home/cmb-06/as/skchoudh/genomes/panTro3/ribotricer_v96_annotation_longest_candidate_orfs.tsv'
#mmul8_ribotricer_index = '/home/cmb-06/as/skchoudh/genomes/Mmul8/ribotricer_v96_annotation_longest_candidate_orfs.tsv'


In [3]:
hg38_index = pd.read_csv(hg38_ribotricer_index, sep='\t')
pantro3_index = pd.read_csv(pantro3_ribotricer_index, sep='\t')

/home/cmb-06/as/skchoudh/software_frozen/anaconda37/envs/riboraptor/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
def parse_ribotricer_index(row):
    chrom = row.chrom
    strand = row.strand
    name = row.ORF_ID
    coordinates = row.coordinate.split(',')
    starts, ends = [], []
    for coordinate in coordinates:
        start, end = coordinate.split('-')
        start = int(start)
        end = int(end)
        # 0-based start
        start = start-1
        starts.append(start)
        ends.append(end)
    bedline = ''
    for start, end in zip(starts, ends):
        score = end-start
        bedline += '{}\t{}\t{}\t{}\t{}\t{}\n'.format(chrom, start, end, name, score, strand)
    return bedline

In [11]:
bedline = ''
for idx, row in hg38_index.iterrows():
    bedline+=parse_ribotricer_index(row)
#hg38_index.head()#[['chrom', 'start', 'end', 'ORF_ID', 'length', 'strand']].head()

In [12]:
with open('/home/cmb-06/as/skchoudh/genomes/hg38/ribotricer_v96_annotation_longest_candidate_orfs.bed', 'w') as fh:
    fh.write(bedline)

In [15]:
!liftOver /home/cmb-06/as/skchoudh/genomes/hg38/ribotricer_v96_annotation_longest_candidate_orfs.bed \
/home/cmb-06/as/skchoudh/liftOver_chains/GRCh38ToPanTro3.over.chain \
/home/cmb-06/as/skchoudh/genomes/hg38/ribotricer_v96_annotation_longest_candidate_orfs.GRCh38ToPanTro3.bed \
/home/cmb-06/as/skchoudh/genomes/hg38/ribotricer_v96_annotation_longest_candidate_orfs.GRCh38ToPanTro3.unmapped.bed

Reading liftover chains
Mapping coordinates


In [16]:
bedline = ''
for idx, row in pantro3_index.iterrows():
    bedline+=parse_ribotricer_index(row)
#hg38_index.head()#[['chrom', 'start', 'end', 'ORF_ID', 'length', 'strand']].head()

In [17]:
with open('/home/cmb-06/as/skchoudh/genomes/panTro3/ribotricer_v96_annotation_longest_candidate_orfs.bed', 'w') as fh:
    fh.write(bedline)